# Figure 2A scripts

In [1]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.metrics import roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import sklearn.impute
import sklearn.ensemble
import scipy.stats
import matplotlib as mpl
import seaborn as sns
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
mpl.rcParams['figure.dpi'] = 250
pd.set_option('display.max_rows', 50)

#load dataset acquired by running the "main_process_68k_pbmc.R" script
df = pd.read_csv("1000_variable_genes.csv", sep=",", header=0, index_col=0)
df = df.drop(["X1"],axis = 1)
df = df.drop(["X2"],axis = 1)
df = df[df.cls_id != "CD4+ T Helper2"]
df.head()

/.mounts/labs/reimandlab/private/users/mslobodyanyuk/anaconda3/envs/myenv/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V992,V993,V994,V995,V996,V997,V998,V999,V1000,cls_id
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,4.25,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.687500,CD8+ Cytotoxic T
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,...,1.021344,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.128063,CD8+/CD45RA+ Naive Cytotoxic
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.977307,0.00,0.0,...,0.977307,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.795764,CD4+/CD25 T Reg
4,0.0,13.615925,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,...,1.512881,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.051522,CD19+ B
5,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,...,1.031949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.383387,CD4+/CD25 T Reg


In [ ]:
#split data from class labels and binarize expression.
X = df.iloc[:,0:1000]
Y = df.cls_id
X[X > 0] = 1 
X[X <= 0] = 0

#provide the ENSG gene id for each feature.
df_genes = pd.read_csv("gene_names_scRNAseq_rf.csv", sep = ",", header = 0, index_col = 0)
X.columns = list(df_genes.x)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, train_size = 0.8)

In [ ]:
#logistic regression 5-fold cross-validation
performance = ['precision_macro', 'recall_macro', 'f1_macro', 'accuracy']
logistic = LogisticRegression(solver='sag', max_iter=1000, multi_class = "multinomial")
logistic_scores = cross_validate(logistic, X, Y, cv=5, scoring=performance)

logistic_dict = {"Precision" : logistic_scores['test_precision_macro'],"Recall" : logistic_scores['test_recall_macro'], "F1":logistic_scores['test_f1_macro'],
                "Accuracy" : logistic_scores['test_accuracy'] }

print(logistic_dict)

In [ ]:
#MLP 5-fold cross-validation
MLP = MLPClassifier()
MLP_scores = cross_validate(MLP, X, Y, cv=5, scoring=performance)

MLP_dict = {"Precision" : MLP_scores['test_precision_macro'],"Recall" : MLP_scores['test_recall_macro'], "F1":MLP_scores['test_f1_macro'],
                "Accuracy" : MLP_scores['test_accuracy'] }

print(MLP_dict)

In [ ]:
#Random forest 5-fold cross-validation
forest = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_features = "sqrt")
forest_scores = cross_validate(forest, X, Y, cv=5, scoring=performance)

forest_dict = {"Precision" : forest_scores['test_precision_macro'],"Recall" : forest_scores['test_recall_macro'], "F1":forest_scores['test_f1_macro'],
                "Accuracy" : forest_scores['test_accuracy'] }

print(forest_dict)

In [ ]:
#xgboost 5-fold cross-validation
xg_cl = xgb.XGBClassifier(
    objective="multi:softmax",
    num_class=10,
    n_estimators=10,
    max_depth=6,
    learning_rate=0.3,
    reg_lambda=1.0,
    reg_alpha=0.0
)
xgb_scores = cross_validate(xg_cl, X, Y, cv=5, scoring=performance)

xgb_dict = {"Precision" : xgb_scores['test_precision_macro'],"Recall" : xgb_scores['test_recall_macro'], "F1":xgb_scores['test_f1_macro'],
                "Accuracy" : xgb_scores['test_accuracy'] }

print(xgb_dict)

In [ ]:
classification_df = pd.DataFrame({"metric": ["Precision","Recall","F1","Accuracy"],"logistic_mean" : [np.mean(v) for k,v in logistic_dict.items()],
                          "logistic_sd" : [np.std(v) for v in logistic_dict.values()],
                          "MLP_mean" : [np.mean(v) for k,v in MLP_dict.items()],
                          "MLP_sd" : [np.std(v) for v in MLP_dict.values()],
                         "random_forest_mean" : [np.mean(v) for k,v in forest_dict.items()],
                          "random_forest_sd" : [np.std(v) for v in forest_dict.values()],
                         "xgb_mean" : [np.mean(v) for k,v in xgb_dict.items()],
                          "xgb_sd" : [np.std(v) for v in xgb_dict.values()]})
print(classification_df)
classification_df.to_csv("performance_figure2A.tsv", sep='\t')

In [ ]:
# Figure 2A bar graph:
ind = np.arange(4)
fig, ax = plt.subplots(figsize = (25, 10))

log_bar = ax.bar(ind - 0.1, classification_df.logistic_mean, yerr = classification_df.logistic_sd,alpha=1,width = 0.2,capsize = 10, color = '#66c2a5',align = 'center')
mlp_bar = ax.bar(ind + 0.1 , classification_df.MLP_mean,yerr = classification_df.MLP_sd,alpha=1, width = 0.2,capsize = 10, color = '#fc8d62', align = 'center')
rf_bar = ax.bar(ind + 0.3, classification_df.random_forest_mean,yerr = classification_df.random_forest_sd,alpha=1, width = 0.2,capsize = 10, color = '#8da0cb',align = 'center')
xgb_bar = ax.bar(ind + 0.5, classification_df.xgb_mean,yerr = classification_df.xgb_sd,alpha=1, width = 0.2,capsize = 10, color = '#e78ac3', align = 'center')
ax.set_ylabel("Model Performance", size = 36)
ax.set_ylim([0.60,1])
ax.set_xlabel("Metric", size = 36)
ax.set_xticks(ind + 0.2,labels = classification_df.metric)
ax.set_xticklabels(classification_df.metric,rotation = 0)
ax.tick_params(axis='both', labelsize=28)
ax.legend((log_bar[0], mlp_bar[0],rf_bar[0],xgb_bar[0]), ('Logistic Regression','MLP','Random Forest', 'XGB'),prop={'size': 22}, bbox_to_anchor=(1.23, 1), loc='upper right')
ax.yaxis.grid(True)
fig.suptitle('Cell classification performance using four ML methods', fontsize=36)
ax.xaxis.labelpad = 20
ax.yaxis.labelpad = 20
vis = ax.get_figure()
vis.savefig("fig2A_performance.pdf", bbox_inches='tight')

# Figure 2B Confusion Matrices

In [ ]:
# re-run logistic regression to acquire the predicted labels
logistic = LogisticRegression(solver='sag', max_iter=1000, multi_class = "multinomial")
logistic.fit(X_train,Y_train)
logistic_pred = logistic.predict(X_test)

# confusion matrix
matrix = confusion_matrix(Y_test,logistic_pred, labels=np.unique(logistic_pred))
matrix = matrix/matrix.sum(axis=1, keepdims=True)*1
df_matrix = pd.DataFrame(matrix)
df_matrix.columns = np.unique(logistic_pred)
df_matrix.index = np.unique(logistic_pred)

sns.set(font_scale=0.65)
ax = sns.heatmap(df_matrix,cmap ="YlGnBu",annot = True,vmin = 0, vmax = 1,linewidth=0.5,annot_kws={"size": 6})
plt.show()
vis = ax.get_figure()
vis.savefig("confusionmtx_logistic.pdf", bbox_inches='tight')

In [ ]:
# re-run MLP to acquire the predicted labels
MLP = MLPClassifier()
MLP.fit(X_train,Y_train)
mlp_pred = MLP.predict(X_test)

# confusion matrix
matrix = confusion_matrix(Y_test,mlp_pred, labels=np.unique(mlp_pred))
matrix = matrix/matrix.sum(axis=1, keepdims=True)*1
df_matrix = pd.DataFrame(matrix)
df_matrix.columns = np.unique(mlp_pred)
df_matrix.index = np.unique(mlp_pred)

sns.set(font_scale=0.65)
ax = sns.heatmap(df_matrix,cmap ="YlGnBu",annot = True,vmin = 0, vmax = 1,linewidth=0.5,annot_kws={"size": 6})
plt.show()
vis = ax.get_figure()
vis.savefig("confusionmtx_mlp.pdf", bbox_inches='tight')

In [ ]:
# re-run random forest to acquire the predicted labels
forest = RandomForestClassifier(n_estimators = 500, criterion = "gini", max_features = "sqrt")
forest.fit(X_train,Y_train)
rf_pred = forest.predict(X_test)

# confusion matrix
matrix = confusion_matrix(Y_test,rf_pred, labels=np.unique(rf_pred))
matrix = matrix/matrix.sum(axis=1, keepdims=True)*1
df_matrix = pd.DataFrame(matrix)
df_matrix.columns = np.unique(rf_pred)
df_matrix.index = np.unique(rf_pred)

sns.set(font_scale=0.65)
ax = sns.heatmap(df_matrix,cmap ="YlGnBu",annot = True,vmin = 0, vmax = 1,linewidth=0.5,annot_kws={"size": 6})
plt.show()
vis = ax.get_figure()
vis.savefig("confusionmtx_rf.pdf", bbox_inches='tight')


In [ ]:
# re-run xgboost to acquire the predicted labels
xg_cl = xgb.XGBClassifier(
    objective="multi:softmax",
    num_class=10,
    n_estimators=10,
    max_depth=6,
    learning_rate=0.3,
    reg_lambda=1.0,
    reg_alpha=0.0
)
xg_cl.fit(X_train,Y_train)
xgb_pred = xg_cl.predict(X_test)

# confusion matrix
matrix = confusion_matrix(Y_test,xgb_pred, labels=np.unique(xgb_pred))
matrix = matrix/matrix.sum(axis=1, keepdims=True)*1
df_matrix = pd.DataFrame(matrix)
df_matrix.columns = np.unique(xgb_pred)
df_matrix.index = np.unique(xgb_pred)

sns.set(font_scale=0.65)
ax = sns.heatmap(df_matrix,cmap ="YlGnBu",annot = True,vmin = 0, vmax = 1,linewidth=0.5,annot_kws={"size": 6})
plt.show()
vis = ax.get_figure()
vis.savefig("confusionmtx_xgb.pdf", bbox_inches='tight')

# Figure 3 Feature Importance using Logistic Regression

In [ ]:
#on an internet node
import mygene

mg = mygene.MyGeneInfo()
gene_symbols = mg.querymany(X.columns, scopes = 'ensembl.gene',fields = 'symbol',species = 'human')
gene_symbols

y_df = pd.DataFrame(gene_symbols)
y_df.to_csv("genes_1000_symbols.csv", sep=',')

In [ ]:
#for each cell-type compute the top 20 most positive and negative gene weights in logistic regression model
genes = pd.read_csv("genes_1000_symbols.csv",sep = ",", header = 0)
genes['symbol'] = genes['symbol'].fillna(genes['query'])

for i in range(10):
    logistic_df = pd.DataFrame({"genes":genes.symbol, "coefficient" : logistic.coef_[i]})
    logistic_df = logistic_df.sort_values('coefficient',ascending = False)
    logistic_df = pd.concat([logistic_df.head(20),logistic_df.tail(20)])
    
    ind = np.arange(40)
    fig, ax = plt.subplots(figsize = (25, 10))
    colours = ['#d53e4f' if (x < 0) else '#1b7837' for x in logistic_df.coefficient]
    logistic_bar = ax.bar(ind, logistic_df.coefficient,alpha=1.0,width = 0.6,capsize = 10, color = colours)
    ax.set_ylabel("Feature Importance (AU)", size = 38)
    ax.set_xlabel("Gene", size = 38)
    ax.set_xticks(ind,labels = logistic_df.genes)
    ax.set_xticklabels(logistic_df.genes,rotation = 90)
    ax.tick_params(axis='both', labelsize=34)
    ax.yaxis.grid(True)
    ax.xaxis.labelpad = 20
    ax.yaxis.labelpad = 20

    ax.patch.set_edgecolor('black')  
    ax.patch.set_linewidth('2') 
    fig.suptitle(np.unique(logistic_pred)[i], fontsize=38)
    vis = ax.get_figure()
    vis.savefig(str(i) + ".pdf", bbox_inches='tight')